In [11]:
from importlib import reload ,import_module
import module.utilize as utilize
import module.multiVariant as multiVariant
import module.singleVariant as singleVariant
import module.multiHistogramBase as multiHistogramBase
import numpy as np
from numba import njit,jit, float32
import module.singleVariantCopulaBase as CopulaBase
from tqdm import tqdm
import time
from multiprocessing import Pool
from sklearn.metrics import root_mean_squared_error
import cupy as cp
import module.multiHistogramSparse as multiHistogramSparse
reload(utilize)
reload(multiVariant)
reload(singleVariant)
reload(multiHistogramBase)
reload(CopulaBase)
reload(multiHistogramSparse)
import module.ananlysisFuncion as analysisFunction


startTime=time.time()

#attribute_names=np.array(["phi_grav","particle_mass_density","zmom","ymom"])
attribute_names=np.array(["phi_grav","particle_mass_density"])
incremental_number=300
all_ensamble_data=np.empty([0,incremental_number,64,64,64])

for name in attribute_names:
    data=utilize.readFiles(name,incremental_number)
    data=data.reshape(1,incremental_number,64,64,64)
    all_ensamble_data=np.append(all_ensamble_data,data,axis=0)

#print(all_ensamble_data.shape)
#print(all_ensamble_data[0].shape)
covBlockSize=5
dataBlockSize=5
binsNumber=128
sizeZ=20
sizeY=20
sizeX=20
minMaxBlockSize=2
isMinMax=False

conditions=np.array([[0,1e5],[3e10,5e10]])

gtModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=1,binsNumber=binsNumber)
gtModel.fit()

result=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
oursResult=np.zeros([sizeZ,sizeY,sizeX],dtype=np.float32)
oursModel=multiVariant.multiDistCopula3D.load(f"Nyx_{attribute_names.shape[0]}varaibles_{incremental_number}members_128Bins_dBlock5_cBlock2_new")
with tqdm(total=sizeZ*sizeY*sizeX, desc="總進度") as pbar:
    for idx in range(sizeZ * sizeY * sizeX):
        
        z = idx // (sizeY * sizeX)
        y = (idx // sizeX) % sizeY
        x = idx % sizeX      

        samples=gtModel.sampleByPos(z,y,x)
        
        prob=analysisFunction.probability_in_range_numba(samples,conditions)

        result[z,y,x]=prob

        
        samples=oursModel.sampleByPos(z,y,x)

        prob=analysisFunction.probability_in_range_numba(samples,conditions)

        oursResult[z,y,x]=prob
        pbar.update(1)

result.tofile("newProb.bin")
oursResult.tofile("ous.bin")

總進度: 100%|██████████| 8000/8000 [01:01<00:00, 130.59it/s]


In [6]:
import numpy as np

a=np.array([[3,4,5],[6,7,8]])
b=a.T
print(a)
print(b)

[[3 4 5]
 [6 7 8]]
[[3 6]
 [4 7]
 [5 8]]
